# Modèle 1 : Implémentation d'un Sentence Transformers (Architecture 1)

## Intallation et import

In [8]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 5.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [1]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Chardement des données

L'objectif de cette partie est de lire les fichiers .csv contenu dans le dossier data, d'en extraire les données pertinentes et de renvoyer un dataset propre contenant [num_solution, données_pertinentes_concaténées].

Dans un premier temps nous avons choisi de garder ces données sur nos solutions : 'titre', 'definition', 'application', 'bilan_energie'.

**Il peut être pertinent de tester d'inclure plus de données sur les technologies par exemple.**

In [2]:
def load_and_merge_data(csv_file = '../data/solutions.csv'):
    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)
    # Renommer les colonnes
    df.columns = ['id_solution', 'categorie', 'texte']
    # Filtrer les lignes pour les catégories spécifiées
    categories_specifiees = [1, 2, 5, 6]
    df_filtre = df[df['categorie'].isin(categories_specifiees)]
    # Pivoter les données pour obtenir les colonnes 'titre', 'definition', 'application' et 'bilan énergie'
    df_pivot = df_filtre.pivot(index='id_solution', columns='categorie', values='texte').reset_index()
    # Renommer les colonnes
    df_pivot.columns = ['id_solution', 'titre', 'definition', 'application', 'bilan_energie']

    # Ajouter un point à la fin des colonnes si nécessaire
    colonnes = ['titre', 'definition', 'application', 'bilan_energie']
    for col in colonnes:
        df_pivot[col] = df_pivot[col].apply(lambda x: x.strip() + '.' if isinstance(x, str) and x.strip()[-1] != '.' else x.strip() if isinstance(x, str) else x)
    
    # Gérer les valeurs NaN lors de la fusion des colonnes
    def combine_text(row):
        text_parts = [row[col] for col in colonnes if pd.notnull(row[col])]
        return ' '.join(text_parts)
    
    # Appliquer la fonction pour créer la nouvelle colonne 'text'
    df_pivot['text'] = df_pivot.apply(combine_text, axis=1)

    # Sélectionner uniquement les colonnes 'id_solution' et 'text'
    df_final = df_pivot[['id_solution', 'text']]
    
    return df_final

In [3]:
# Affachage des données chargées
df_solutions = load_and_merge_data()
print(df_solutions)

     id_solution                                               text
0              2  Installation frigorifique négative de type cas...
1              4  Arrêt des compresseurs le week end et lorsque ...
2              5  Régulation non électronique sur un compresseur...
3              8  Vérification du dimensionnement et remplacemen...
4              9  Zone neutre entre les températures de consigne...
..           ...                                                ...
536         1555  Triple vitrage. Le triple vitrage est constitu...
537         1557  Vitrage à store intégré. Cette solution se com...
538         1559  Optimisation de la phase poste minage (explosi...
539         1560  Arrêt des locotracteurs lors de leur chargemen...
540         1561  Récupération de l’énergie potentielle de desce...

[541 rows x 2 columns]


## Pré-traitement des données

Pour le pré-traitemment :
- Nettoyer HTML tags
- Remplacer "&nbsp;." par rien. Cette balise correspond à une image dans la database.
- Enlever les accents.
- Retirer les numéros.
- Tokenization
- Lemmatization
- Removing Stopwords
- Lowercase

In [4]:
# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

def pre_processing(texte):
    # Nettoyer HTML Tags
    texte = BeautifulSoup(texte, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    texte = re.sub(r'&nbsp;\.', '', texte)

    # Accents
    texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

    # Retirer les numéros
    texte = re.sub(r'\b\d+\b', '', texte)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(texte)
    phrases_propres = []
    for phrase in doc.sents:
        tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
        phrase_propre = ' '.join(tokens)
        if phrase_propre:
            phrases_propres.append(phrase_propre + ".")  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(phrases_propres)

    return cleaned_text

In [5]:
# Affichage des données nettoyées de la solution id_solution
id_solution = 2

text_brut = df_solutions[df_solutions['id_solution'] == id_solution]['text'].iloc[0]
clean_text = pre_processing(text_brut)

print("Text brut :\n", text_brut, "\nText pré-traité :\n",clean_text)

Text brut :
 Installation frigorifique négative de type cascade utilisant du CO2. Mise en place d'une installation frigorifique négative de type cascade utilisant du CO2 comme fluide frigorigène. Pour être éligible à CEE, la mise en place doit être effectuée par un professionnel et appliquée dans des locaux de commerce de distribution alimentaire de surface de vente inférieure à 5000 m². Comparé aux autres fluides frigorigènes, le CO2 est un fluide. 
Text pré-traité :
 installation frigorifique negativ type cascade utiliser co2. mise place installation frigorifique negativ type cascade utiliser co2 fluide frigorigene. eligibl cee mise place effectuee professionnel appliquee local commerce distribution alimentaire surface vente inferieure m. compare fluide frigorigene co2 fluide.


In [6]:
# Appliquons le traitement à la colonne text de notre df_solutions
df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing)

/tmp/ipykernel_6856/1761720504.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


## Choix du modèle

Nous vons choisi d'implémenter le modèle **paraphrase-multilingual-MiniLM-L12-v2** qui est un modèle multi-lingual (+50 languages).
- Max sequence Length : 128 (au dela les mots sont cropés.)
- Dimensions : 384
- Suitable Score Functions: cosine-similarity (util.cos_sim)

Source : https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

**Il serait pertinent d'essayer des modèles spécialisés en Français tel que CamemBert.**

In [18]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

## Calcul de l'embedding

On définit une fonction *calculate_average_embedding()* qui prend un text en entrée et retourne la moyenne de l'embedding de ses phrases. 

Nous faisons cela afin de ne pas cropper nos données car la max-sequence-length est de 128 sur ce modèle et nos données pouvant être particulièrement longues.

In [7]:
def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

La fonction *genere_embedding()* nous permet de stocker nos embeddings une fois que nous les avons générés. Cela est particulièrement utile car leur génération prend 1"30. Nous ne pouvons pas nous permettre de les regénérer à chaque requette que ferais l'utilisateur.

In [8]:
def genere_embedding(data, output_file, quantize=False, precision="binary"):

    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_average_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_average_embedding)

    
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings
    }
    
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("Les embeddings ont été storer avec succès.")

In [9]:
# Générons notre embedding
genere_embedding(df_solutions, "model1_embeddings.pkl")

NameError: name 'model' is not defined

## Calcul des solutions

La fonction *find_solution()* prend en entrée un texte à comparer, calcul son embedding et retourne une liste contenant ses 10 plus proches [solutions, similarité].

Attention à bien nettoyer le texte avec notre pré-processing avant de le passer à cette fonction.

**Il peut être pertinent de renvoyer toutes les solutions > à une certaine similarité.**

In [15]:
def find_solution(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)
    
    # Convertir la liste de tableaux numpy en un seul tableau numpy
    embeddings_array = np.stack(df_embeddings['text_embedding'].values)

    # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
    similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1), embeddings_array)
    
    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = similarities.flatten()
    
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    
    return solution_list

In [ ]:
# Exemple d'utilisation pour trouver les solutions les plus proches d'un nouveau texte
# ici on a pris le texte correspondant à la solution 3. On doit trouver une similarité de 1.
text_to_compare = "centrale negativ mode booster. installation centrale frigorifique bas moyenne temperatur mode booster consister injecter refoulement compresseur froid negatif aspiration compresseur cycle froid positif. maniere rendement cycle frigorifique bas temperature negatif grandement ameliore condensation bas temperature egal temperature evaporation central positif. solution diminuer quantite tuyauterie utilisee. solution applicable uniquement projet neuf projet refonte total systeme production froid. eligibl production froid niveau temperature. production froid positif temperature consigne 0c temperature evaporation cote fluide -10c exemple. production froid negatif temperature consigne bien inferieur 0c temperature evaporation c exemple. besoin froid positif superieur besoin froid negatif. niveau temperature exempe retrouve secteur activite grande moyenne surface distribution alimentaire gms. industrie agroalimentaire. chimie pharmacie."
solutions = find_solution(text_to_compare, "model1_embeddings.pkl")

# Afficher les solutions les plus proches avec leurs similarités
print("Les solutions les plus proches :")
for solution in solutions:
    print(f"id_solution : {solution[0]}, Similarité : {solution[1]}")

Les solutions les plus proches :
id_solution : 3, Similarité : 1.0000001192092896
id_solution : 42, Similarité : 0.8587038516998291
id_solution : 867, Similarité : 0.829397439956665
id_solution : 19, Similarité : 0.818574845790863
id_solution : 922, Similarité : 0.8174538016319275
id_solution : 35, Similarité : 0.8152481913566589
id_solution : 354, Similarité : 0.8119463920593262
id_solution : 28, Similarité : 0.8111556768417358
id_solution : 918, Similarité : 0.8091707825660706
id_solution : 25, Similarité : 0.8053005933761597


/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/sentence_transformers/util.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  b = torch.tensor(b)


## Quantization

On peut appliquer de la quantization afin de réduire la taille des embeddings stockés de sorte à réduire le temps d'accès à nos données ainsi que la taille du stockage nécessaire sur nos serveurs.

Ici la quantization n'est pas pertinente car on passe d'un fichier embedding de 1.7MB à 459.6KB, on ne gagne pas de temps d'execution et on perd en précision de prédiction. Ce pourrait être utile si le nombre de solutions présents dans la BDD était bien plus grand, peut être à implémenté dans le futur.

In [18]:
genere_embedding(df_solutions, "model1_embeddings_quantized.pkl", quantize=True, precision="binary")

# Exemple d'utilisation pour trouver les solutions les plus proches d'un nouveau texte
text_to_compare = "centrale negativ mode booster. installation centrale frigorifique bas moyenne temperatur mode booster consister injecter refoulement compresseur froid negatif aspiration compresseur cycle froid positif. maniere rendement cycle frigorifique bas temperature negatif grandement ameliore condensation bas temperature egal temperature evaporation central positif. solution diminuer quantite tuyauterie utilisee. solution applicable uniquement projet neuf projet refonte total systeme production froid. eligibl production froid niveau temperature. production froid positif temperature consigne 0c temperature evaporation cote fluide -10c exemple. production froid negatif temperature consigne bien inferieur 0c temperature evaporation c exemple. besoin froid positif superieur besoin froid negatif. niveau temperature exempe retrouve secteur activite grande moyenne surface distribution alimentaire gms. industrie agroalimentaire. chimie pharmacie."
solutions = find_solution(text_to_compare, "model1_embeddings_quantized.pkl", quantize=True)

# Afficher les solutions les plus proches avec leurs similarités
print("Les solutions les plus proches :")
for solution in solutions:
    print(f"id_solution : {solution[0]}, Similarité : {solution[1]}")

Les embeddings ont été storer avec succès.
Les solutions les plus proches :
id_solution : 3, Similarité : 1.0000000000000002
id_solution : 280, Similarité : 0.8338601026755779
id_solution : 26, Similarité : 0.8293075388193698
id_solution : 108, Similarité : 0.8239859183736618
id_solution : 958, Similarité : 0.8224887399894056
id_solution : 354, Similarité : 0.8173533730067889
id_solution : 1119, Similarité : 0.8071937765402742
id_solution : 264, Similarité : 0.7943726247579588
id_solution : 903, Similarité : 0.7862356686070046
id_solution : 824, Similarité : 0.7855407449267934


## Utilisation du modèle

On définit ici le code qui va être executé lorsqu'un utilisateur va faire une requette.

Pour simuler une requette, nous allons charger les données mises à notre disposition par IA Pau (Mail : Exemple de solutions).

On voit que sur les données qu'IA Pau nous a fait parvenir, notre modèle ne semble pas très performant.

**Il serait pertinent de générer notre propre dataset de test afin de pouvoir réaliser des tests plus avancés. De plus nous devons essayer d'autres modèles puis essayer de fine-tuner notre modèle.**

In [12]:
# Malheureusement nous n'avons pas reçu le domaine d'activité correspondant aux 
# requetes avec Kerdos.Et pour certaines requetes nous n'avons pas l'Id_solution non
# plus, dans ce cas il est remplacé par -1.
dataset_test_kerdos = [
    ["Id_solution", "Domaine_activite", "Description"],
    [724, "","C'est quoi la HP flottante ?"],
    [914, "", "Je voudrais dimensionner un panneau solaire."],
    [719, "", "Quel gain pour un variateur de vitesse ?"],
    [-1, "", "J'aimerais avoir une régulation optimisée de mon groupe froid."],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"]
]

In [20]:
# Fonction appelé par notre utilisateur
def model_PAT(secteur, description) :
    # On commence par concaténer notre secteur et notre description.
    text = secteur + ". " + description

    # Ensuite on applique notre pré-processing
    clean_text = pre_processing(text)

    # Ensuite on cherche nos similarités 
    solutions = find_solution(clean_text, "embeddings/paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

    # On return une liste contenant uniquement le numéros des solutions
    id_solutions = []
    for solution in solutions :
        id_solutions.append(solution[0])

    return id_solutions

In [21]:
# On va tester sur notre dataset_test
for i in range(1,len(dataset_test_kerdos)):
    print("--------------------------------------------")
    print("Solution attendue : ", dataset_test_kerdos[i][0])
    print(model_PAT(dataset_test_kerdos[i][1], dataset_test_kerdos[i][2]))

--------------------------------------------
Solution attendue :  724
[724, 102, 722, 823, 1103, 1600, 1613, 1454, 1456, 1455]
--------------------------------------------
Solution attendue :  914
[818, 270, 1486, 858, 914, 1635, 368, 1554, 1631, 1636]
--------------------------------------------
Solution attendue :  719
[1649, 1595, 1445, 1596, 1144, 719, 1533, 1592, 1140, 1653]
--------------------------------------------
Solution attendue :  -1
[1049, 483, 259, 723, 1693, 149, 4, 918, 1130, 9]
--------------------------------------------
Solution attendue :  -1
[1056, 1001, 165, 152, 1058, 209, 153, 145, 154, 1051]


# Test patrice_test_set

In [22]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/patrice_test_set.csv")

def test_accuracy(dataset=df_testset, top_n=1) :
    accuracy = 0
    for i in range(1,len(dataset)):
        predictions = model_PAT("", dataset['Description'][i])
        if (dataset["id_solution"][i] in predictions[:top_n]):
            accuracy += 1/len(dataset)
    return accuracy

In [23]:
test_accuracy()

0.5769230769230769